# V-JEPA 2 Demo Notebook

This tutorial provides an example of how to load the V-JEPA 2 model in vanilla PyTorch and HuggingFace, extract a video embedding, and then predict an action class. For more details about the paper and model weights, please see https://github.com/facebookresearch/vjepa2.

First, let's import the necessary libraries and load the necessary functions for this tutorial.

In [23]:
# Install all required packages
!pip install decord transformers torch torchvision accelerate safetensors huggingface_hub

In [25]:
# Clone the V-JEPA2 repository
!git clone https://github.com/facebookresearch/vjepa2.git
%cd vjepa2

# Install dependencies
!pip install -e .
!pip install decord transformers torch torchvision

Cloning into 'vjepa2'...
remote: Enumerating objects: 310, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 310 (delta 45), reused 33 (delta 33), pack-reused 239 (from 3)
Receiving objects: 100% (310/310), 576.64 KiB | 6.95 MiB/s, done.
Resolving deltas: 100% (141/141), done.
/content/vjepa2/vjepa2
Obtaining file:///content/vjepa2/vjepa2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for vjepa2 (pyproject.toml) ... done
  Created wheel for vjepa2: filename=vjepa2-0.0.1-0.editable-py3-none-any.whl size=2308 sha256=16b8b193286febaf9cb12528ae4fcfd14cc8500cf36f98c59b2d5d67f4d51ecd
  Stored in directory: /tmp/pip-ephem-wheel-cache-qfl8hgop/wheels/11/e3/a7/719966d3487b016995f4f22d87874efacfaa823b420185d973
Successfully built vjepa2
  Attempting unin

In [26]:
import json
import os
import subprocess

import numpy as np
import torch
import torch.nn.functional as F
from decord import VideoReader
from transformers import AutoVideoProcessor, AutoModel

import src.datasets.utils.video.transforms as video_transforms
import src.datasets.utils.video.volume_transforms as volume_transforms
from src.models.attentive_pooler import AttentiveClassifier
from src.models.vision_transformer import vit_giant_xformers_rope

IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)

def load_pretrained_vjepa_pt_weights(model, pretrained_weights):
    # Load weights of the VJEPA2 encoder
    # The PyTorch state_dict is already preprocessed to have the right key names
    pretrained_dict = torch.load(pretrained_weights, weights_only=True, map_location="cpu")["encoder"]
    pretrained_dict = {k.replace("module.", ""): v for k, v in pretrained_dict.items()}
    pretrained_dict = {k.replace("backbone.", ""): v for k, v in pretrained_dict.items()}
    msg = model.load_state_dict(pretrained_dict, strict=False)
    print("Pretrained weights found at {} and loaded with msg: {}".format(pretrained_weights, msg))


def load_pretrained_vjepa_classifier_weights(model, pretrained_weights):
    # Load weights of the VJEPA2 classifier
    # The PyTorch state_dict is already preprocessed to have the right key names
    pretrained_dict = torch.load(pretrained_weights, weights_only=True, map_location="cpu")["classifiers"][0]
    pretrained_dict = {k.replace("module.", ""): v for k, v in pretrained_dict.items()}
    msg = model.load_state_dict(pretrained_dict, strict=False)
    print("Pretrained weights found at {} and loaded with msg: {}".format(pretrained_weights, msg))


def build_pt_video_transform(img_size):
    short_side_size = int(256.0 / 224 * img_size)
    # Eval transform has no random cropping nor flip
    eval_transform = video_transforms.Compose(
        [
            video_transforms.Resize(short_side_size, interpolation="bilinear"),
            video_transforms.CenterCrop(size=(img_size, img_size)),
            volume_transforms.ClipToTensor(),
            video_transforms.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
        ]
    )
    return eval_transform


def get_video():
    vr = VideoReader("sample_video.mp4")
    # choosing some frames here, you can define more complex sampling strategy
    frame_idx = np.arange(0, 128, 2)
    video = vr.get_batch(frame_idx).asnumpy()
    return video


def forward_vjepa_video(model_hf, model_pt, hf_transform, pt_transform):
    # Run a sample inference with VJEPA
    with torch.inference_mode():
        # Read and pre-process the image
        video = get_video()  # T x H x W x C
        video = torch.from_numpy(video).permute(0, 3, 1, 2)  # T x C x H x W
        x_pt = pt_transform(video).cuda().unsqueeze(0)
        x_hf = hf_transform(video, return_tensors="pt")["pixel_values_videos"].to("cuda")
        # Extract the patch-wise features from the last layer
        out_patch_features_pt = model_pt(x_pt)
        out_patch_features_hf = model_hf.get_vision_features(x_hf)

    return out_patch_features_hf, out_patch_features_pt


def get_vjepa_video_classification_results(classifier, out_patch_features_pt):
    SOMETHING_SOMETHING_V2_CLASSES = json.load(open("ssv2_classes.json", "r"))

    with torch.inference_mode():
        out_classifier = classifier(out_patch_features_pt)

    print(f"Classifier output shape: {out_classifier.shape}")

    print("Top 5 predicted class names:")
    top5_indices = out_classifier.topk(5).indices[0]
    top5_probs = F.softmax(out_classifier.topk(5).values[0]) * 100.0  # convert to percentage
    for idx, prob in zip(top5_indices, top5_probs):
        str_idx = str(idx.item())
        print(f"{SOMETHING_SOMETHING_V2_CLASSES[str_idx]} ({prob}%)")

    return

Next, let's download a sample video to the local repository. If the video is already downloaded, the code will skip this step. Likewise, let's download a mapping for the action recognition classes used in Something-Something V2, so we can interpret the predicted action class from our model.

In [27]:
sample_video_path = "sample_video.mp4"
# Download the video if not yet downloaded to local path
if not os.path.exists(sample_video_path):
    video_url = "https://huggingface.co/datasets/nateraw/kinetics-mini/resolve/main/val/bowling/-WH-lxmGJVY_000005_000015.mp4"
    command = ["wget", video_url, "-O", sample_video_path]
    subprocess.run(command)
    print("Downloading video")

# Download SSV2 classes if not already present
ssv2_classes_path = "ssv2_classes.json"
if not os.path.exists(ssv2_classes_path):
    command = [
        "wget",
        "https://huggingface.co/datasets/huggingface/label-files/resolve/d79675f2d50a7b1ecf98923d42c30526a51818e2/"
        "something-something-v2-id2label.json",
        "-O",
        "ssv2_classes.json",
    ]
    subprocess.run(command)
    print("Downloading SSV2 classes")

Now, let's load the models in both vanilla Pytorch as well as through the HuggingFace API. Note that HuggingFace API will automatically load the weights through `from_pretrained()`, so there is no additional download required for HuggingFace.

To download the PyTorch model weights, use wget and specify your preferred target path. See the README for the model weight URLs.
E.g.
```
wget https://dl.fbaipublicfiles.com/vjepa2/vitg-384.pt -P YOUR_DIR
```
Then update `pt_model_path` with `YOUR_DIR/vitg-384.pt`. Also note that you have the option to use `torch.hub.load`.

In [28]:
# HuggingFace model repo name
hf_model_name = (
    "facebook/vjepa2-vitg-fpc64-384"  # Replace with your favored model, e.g. facebook/vjepa2-vitg-fpc64-384
)
# Path to local PyTorch weights
#pt_model_path = "YOUR_MODEL_PATH"

# Initialize the HuggingFace model, load pretrained weights
model_hf = AutoModel.from_pretrained(hf_model_name)
model_hf.cuda().eval()

# Build HuggingFace preprocessing transform
hf_transform = AutoVideoProcessor.from_pretrained(hf_model_name)
img_size = hf_transform.crop_size["height"]  # E.g. 384, 256, etc.

# Initialize the PyTorch model, load pretrained weights
#model_pt = vit_giant_xformers_rope(img_size=(img_size, img_size), num_frames=64)
#model_pt.cuda().eval()
#load_pretrained_vjepa_pt_weights(model_pt, pt_model_path)

### Can also use torch.hub to load the model
# model_pt, _ = torch.hub.load('facebookresearch/vjepa2', 'vjepa2_vit_giant_384')
# model_pt.cuda().eval()

# Build PyTorch preprocessing transform
pt_video_transform = build_pt_video_transform(img_size=img_size)

Now we can run the encoder on the video to get the patch-wise features from the last layer of the encoder. To verify that the HuggingFace and PyTorch models are equivalent, we will compare the values of the features.

In [29]:
# Inference on video to get the patch-wise features
out_patch_features_hf, out_patch_features_pt = forward_vjepa_video(
    model_hf, model_pt, hf_transform, pt_video_transform
)

print(
    f"""
    Inference results on video:
    HuggingFace output shape: {out_patch_features_hf.shape}
    PyTorch output shape:     {out_patch_features_pt.shape}
    Absolute difference sum:  {torch.abs(out_patch_features_pt - out_patch_features_hf).sum():.6f}
    Close: {torch.allclose(out_patch_features_pt, out_patch_features_hf, atol=1e-3, rtol=1e-3)}
    """
)

/usr/lib/python3.12/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)



    Inference results on video:
    HuggingFace output shape: torch.Size([1, 18432, 1408])
    PyTorch output shape:     torch.Size([1, 18432, 1408])
    Absolute difference sum:  54907976.000000
    Close: False
    


In [ ]:
# Download the attentive probe classifier weights for Something-Something V2
!wget https://dl.fbaipublicfiles.com/vjepa2/evals/ssv2-vitg-384-64x2x3.pt -P /content/classifiers

# Set the classifier path
classifier_model_path = "/content/classifiers/ssv2-vitg-384-64x2x3.pt"

# Initialize the classifier
# Note: Since you're using HuggingFace model, use model_hf.config.hidden_size instead of model_pt.embed_dim
classifier = (
    AttentiveClassifier(
        embed_dim=model_hf.config.hidden_size,  # or use 1408 directly
        num_heads=16,
        depth=4,
        num_classes=174
    ).cuda().eval()
)
load_pretrained_vjepa_classifier_weights(classifier, classifier_model_path)

# Get classification results (use HuggingFace features)
get_vjepa_video_classification_results(classifier, out_patch_features_hf)

Great! Now we know that the features from both models are equivalent. Now let's run a pretrained attentive probe classifier on top of the extracted features, to predict an action class for the video. Let's use the Something-Something V2 probe. Note that the repository also includes attentive probe weights for other evaluations such as EPIC-KITCHENS-100 and Diving48.

To download the attentive probe weights, use wget and specify your preferred target path. E.g. `wget https://dl.fbaipublicfiles.com/vjepa2/evals/ssv2-vitg-384-64x2x3.pt -P YOUR_DIR`

Then update `classifier_model_path` with `YOUR_DIR/ssv2-vitg-384-64x2x3.pt`.

In [30]:
# Initialize the classifier
#classifier_model_path = "YOUR_ATTENTIVE_PROBE_PATH"

# Download the attentive probe classifier weights for Something-Something V2
!wget https://dl.fbaipublicfiles.com/vjepa2/evals/ssv2-vitg-384-64x2x3.pt -P /content/classifiers

# Set the classifier path
classifier_model_path = "/content/classifiers/ssv2-vitg-384-64x2x3.pt"

classifier = (
    AttentiveClassifier(embed_dim=model_pt.embed_dim, num_heads=16, depth=4, num_classes=174).cuda().eval()
)
load_pretrained_vjepa_classifier_weights(classifier, classifier_model_path)

# Get classification results
get_vjepa_video_classification_results(classifier, out_patch_features_pt)

--2026-01-29 02:36:28--  https://dl.fbaipublicfiles.com/vjepa2/evals/ssv2-vitg-384-64x2x3.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.171.22.13, 3.171.22.33, 3.171.22.118, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.171.22.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 373996872 (357M) [application/vnd.snesdev-page-table]
Saving to: ‘/content/classifiers/ssv2-vitg-384-64x2x3.pt.2’

ssv2-vitg-384-64x2x 100%[===================>] 356.67M   294MB/s    in 1.2s    

2026-01-29 02:36:29 (294 MB/s) - ‘/content/classifiers/ssv2-vitg-384-64x2x3.pt.2’ saved [373996872/373996872]

Pretrained weights found at /content/classifiers/ssv2-vitg-384-64x2x3.pt and loaded with msg: <All keys matched successfully>
Classifier output shape: torch.Size([1, 174])
Top 5 predicted class names:


/tmp/ipython-input-2987202713.py:85: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  top5_probs = F.softmax(out_classifier.topk(5).values[0]) * 100.0  # convert to percentage


Moving [something] away from the camera (25.095457077026367%)
Burying [something] in [something] (19.680456161499023%)
Pretending to pick [something] up (19.282258987426758%)
Putting [something] on a surface (18.631010055541992%)
Pretending to turn [something] upside down (17.310815811157227%)


The video features a man putting a bowling ball into a tube, so the predicted action of "Putting [something] into [something]" makes sense!

This concludes the tutorial. Please see the README and paper for full details on the capabilities of V-JEPA 2 :)

Testing on our own video

In [32]:
# Direct path to your video file
video_path = "/content/episode_000000.mp4"

print(f"\nProcessing video: {video_path}")

# Load video
video_reader = VideoReader(video_path)
frames = [frame.asnumpy() for frame in video_reader]

# Preprocess with HuggingFace
inputs = hf_transform(frames, return_tensors="pt")
inputs = {k: v.cuda() for k, v in inputs.items()}

# Get features
with torch.no_grad():
    outputs = model_hf(**inputs)
    out_features_custom = outputs.last_hidden_state

print(f"Extracted features shape: {out_features_custom.shape}")

# Run classification
get_vjepa_video_classification_results(classifier, out_features_custom)


Processing video: /content/episode_000000.mp4


DECORDError: [02:38:41] /github/workspace/src/video/video_reader.cc:151: Check failed: st_nb >= 0 (-1128613112 vs. 0) ERROR cannot find video stream with wanted index: -1

Debugging why it can't read video

In [33]:
# Test and process video with better error handling
video_path = "/content/episode_000000.mp4"

print(f"Processing video: {video_path}")

# First, let's check if the file exists and get info
import os
print(f"File exists: {os.path.exists(video_path)}")
print(f"File size: {os.path.getsize(video_path) / (1024*1024):.2f} MB")

# Try to get video info using ffmpeg
!ffprobe -v error -show_entries stream=codec_name,width,height,r_frame_rate -of default=noprint_wrappers=1 "{video_path}"

# Try decord with specific context
try:
    from decord import VideoReader, cpu
    video_reader = VideoReader(video_path, ctx=cpu(0))
    print(f"Video loaded successfully!")
    print(f"Number of frames: {len(video_reader)}")

    # Sample frames instead of loading all
    num_frames = min(64, len(video_reader))  # V-JEPA2 uses 64 frames
    indices = list(range(0, len(video_reader), len(video_reader) // num_frames))[:num_frames]
    frames = video_reader.get_batch(indices).asnumpy()

    print(f"Extracted {len(frames)} frames")

except Exception as e:
    print(f"Decord failed: {e}")
    print("\nTrying alternative method with OpenCV...")

    # Fallback to OpenCV
    import cv2
    cap = cv2.VideoCapture(video_path)
    frames = []

    while len(frames) < 64 and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()
    print(f"Extracted {len(frames)} frames with OpenCV")

# Continue with processing
if len(frames) > 0:
    # Preprocess with HuggingFace
    inputs = hf_transform(frames, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}

    # Get features
    with torch.no_grad():
        outputs = model_hf(**inputs)
        out_features_custom = outputs.last_hidden_state

    print(f"Extracted features shape: {out_features_custom.shape}")

    # Run classification
    get_vjepa_video_classification_results(classifier, out_features_custom)
else:
    print("Failed to extract frames from video")

Processing video: /content/episode_000000.mp4
File exists: True
File size: 1.75 MB
codec_name=av1
width=640
height=480
r_frame_rate=30/1
Decord failed: [02:40:09] /github/workspace/src/video/video_reader.cc:151: Check failed: st_nb >= 0 (-1128613112 vs. 0) ERROR cannot find video stream with wanted index: -1

Trying alternative method with OpenCV...
Extracted 0 frames with OpenCV
Failed to extract frames from video


In [34]:
# Convert AV1 video to H.264 format that decord can read
video_path = "/content/episode_000000.mp4"
converted_path = "/content/episode_000000_converted.mp4"

print("Converting video to compatible format...")
!ffmpeg -i "{video_path}" -c:v libx264 -pix_fmt yuv420p -c:a aac "{converted_path}" -y

print(f"\nProcessing converted video: {converted_path}")

# Now try loading the converted video
from decord import VideoReader, cpu

try:
    video_reader = VideoReader(converted_path, ctx=cpu(0))
    print(f"Video loaded successfully!")
    print(f"Number of frames: {len(video_reader)}")

    # Sample frames (V-JEPA2 typically uses 64 frames)
    num_frames = min(64, len(video_reader))
    indices = list(range(0, len(video_reader), len(video_reader) // num_frames))[:num_frames]
    frames = video_reader.get_batch(indices).asnumpy()

    print(f"Extracted {len(frames)} frames, shape: {frames.shape}")

    # Preprocess with HuggingFace
    inputs = hf_transform(frames, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}

    # Get features
    with torch.no_grad():
        outputs = model_hf(**inputs)
        out_features_custom = outputs.last_hidden_state

    print(f"Extracted features shape: {out_features_custom.shape}")

    # Run classification
    get_vjepa_video_classification_results(classifier, out_features_custom)

except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Converting video to compatible format...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab 

/tmp/ipython-input-2987202713.py:85: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  top5_probs = F.softmax(out_classifier.topk(5).values[0]) * 100.0  # convert to percentage


Holding [something] (35.198081970214844%)
Covering [something] with [something] (18.10696029663086%)
Pouring [something] into [something] (17.14968490600586%)
Moving [something] down (16.28497886657715%)
Holding [something] over [something] (13.260290145874023%)


Visualizing labels availble in something to something dataset.
Link: https://www.qualcomm.com/developer/software/something-something-v-2-dataset

In [37]:
# The SSv2 labels are embedded in the dataset

ssv2_labels = {
    0: "Approaching [something] with your camera",
    1: "Attaching [something] to [something]",
    2: "Bending [something] so that it deforms",
    3: "Bending [something] until it breaks",
    4: "Burying [something] in [something]",
    5: "Closing [something]",
    6: "Covering [something] with [something]",
    7: "Digging [something] out of [something]",
    8: "Dropping [something] behind [something]",
    9: "Dropping [something] in front of [something]",
    10: "Dropping [something] into [something]",
    11: "Dropping [something] next to [something]",
    12: "Dropping [something] onto [something]",
    13: "Failing to put [something] into [something] because [something] does not fit",
    14: "Folding [something]",
    15: "Hitting [something] with [something]",
    16: "Holding [something]",
    17: "Holding [something] behind [something]",
    18: "Holding [something] in front of [something]",
    19: "Holding [something] next to [something]",
    20: "Holding [something] over [something]",
    21: "Laying [something] on the table on its side, not upright",
    22: "Letting [something] roll along a flat surface",
    23: "Letting [something] roll down a slanted surface",
    24: "Letting [something] roll up a slanted surface, so it rolls back down",
    25: "Lifting a surface with [something] on it but not enough for it to slide down",
    26: "Lifting a surface with [something] on it until it starts sliding down",
    27: "Lifting up one end of [something without letting it drop down]",
    28: "Lifting up one end of [something], then letting it drop down",
    29: "Moving [something] across a surface until it falls down",
    30: "Moving [something] across a surface without it falling down",
    31: "Moving [something] and [something] away from each other",
    32: "Moving [something] and [something] closer to each other",
    33: "Moving [something] and [something] so they collide with each other",
    34: "Moving [something] and [something] so they pass each other",
    35: "Moving [something] away from [something]",
    36: "Moving [something] away from the camera",
    37: "Moving [something] closer to [something]",
    38: "Moving [something] down",
    39: "Moving [something] towards the camera",
    40: "Moving [something] up",
    41: "Opening [something]",
    42: "Picking [something] up",
    43: "Piling [something] up",
    44: "Plugging [something] into [something]",
    45: "Plugging [something] into [something] but pulling it right out as you remove your hand",
    46: "Poking [something] so lightly that it doesn't or almost doesn't move",
    47: "Poking [something] so that it falls over",
    48: "Poking [something] so that it slightly moves",
    49: "Poking [something] so that it spins around",
    50: "Poking a hole into [something soft]",
    51: "Poking a hole into some substance",
    52: "Poking a stack of [something] so the stack collapses",
    53: "Poking a stack of [something] without the stack collapsing",
    54: "Pouring [something] into [something]",
    55: "Pouring [something] into [something] until it overflows",
    56: "Pouring [something] onto [something]",
    57: "Pouring [something] out of [something]",
    58: "Pretending or failing to wipe [something] off of [something]",
    59: "Pretending or trying and failing to twist [something]",
    60: "Pretending to be tearing [something that is not tearable]",
    61: "Pretending to close [something] without actually closing it",
    62: "Pretending to open [something] without actually opening it",
    63: "Pretending to pick [something] up",
    64: "Pretending to poke [something]",
    65: "Pretending to pour [something] out of [something], but [something] is empty",
    66: "Pretending to put [something] behind [something]",
    67: "Pretending to put [something] into [something]",
    68: "Pretending to put [something] next to [something]",
    69: "Pretending to put [something] on a surface",
    70: "Pretending to put [something] onto [something]",
    71: "Pretending to scoop [something] up with [something]",
    72: "Pretending to spread air onto [something]",
    73: "Pretending to sprinkle air onto [something]",
    74: "Pretending to squeeze [something]",
    75: "Pretending to take [something] from [somewhere]",
    76: "Pretending to take [something] out of [something]",
    77: "Pretending to throw [something]",
    78: "Pretending to turn [something] upside down",
    79: "Pulling [something] from behind of [something]",
    80: "Pulling [something] from left to right",
    81: "Pulling [something] from right to left",
    82: "Pulling [something] onto [something]",
    83: "Pulling [something] out of [something]",
    84: "Pulling two ends of [something] so that it gets stretched",
    85: "Pulling two ends of [something] so that it separates into two pieces",
    86: "Pushing [something] from left to right",
    87: "Pushing [something] from right to left",
    88: "Pushing [something] off of [something]",
    89: "Pushing [something] onto [something]",
    90: "Pushing [something] so it spins",
    91: "Pushing [something] so that it almost falls off but doesn't",
    92: "Pushing [something] so that it falls off the table",
    93: "Pushing [something] so that it slightly moves",
    94: "Pushing [something] with [something]",
    95: "Putting [number of] [something] onto [something]",
    96: "Putting [something] and [something] on the table",
    97: "Putting [something] behind [something]",
    98: "Putting [something] in front of [something]",
    99: "Putting [something] into [something]",
    100: "Putting [something] next to [something]",
    101: "Putting [something] on a flat surface without letting it roll",
    102: "Putting [something] on a surface",
    103: "Putting [something] on the edge of [something] so it is not supported and falls down",
    104: "Putting [something] onto [something]",
    105: "Putting [something] onto [something] else that cannot support it so it falls down",
    106: "Putting [something] onto a slanted surface but it doesn't glide down",
    107: "Putting [something] similar to other things that are already on the table",
    108: "Putting [something] that can't roll onto a slanted surface, so it slides down",
    109: "Putting [something] that can't roll onto a slanted surface, so it stays where it is",
    110: "Putting [something] that cannot actually stand upright upright on the table, so it falls on its side",
    111: "Putting [something], [something] and [something] on the table",
    112: "Putting [something] upright on the table, so it falls on its side",
    113: "Putting [something] underneath [something]",
    114: "Removing [something], revealing [something] behind",
    115: "Rolling [something] on a flat surface",
    116: "Scooping [something] up with [something]",
    117: "Showing [something] behind [something]",
    118: "Showing [something] next to [something]",
    119: "Showing [something] on top of [something]",
    120: "Showing [something] to the camera",
    121: "Showing that [something] is empty",
    122: "Showing that [something] is inside [something]",
    123: "Something being deflected from [something]",
    124: "Spilling [something] behind [something]",
    125: "Spilling [something] next to [something]",
    126: "Spilling [something] onto [something]",
    127: "Spinning [something] so it continues spinning",
    128: "Spinning [something] that quickly stops spinning",
    129: "Spreading [something] onto [something]",
    130: "Sprinkling [something] onto [something]",
    131: "Squeezing [something]",
    132: "Stacking [number of] [something]",
    133: "Stuffing [something] into [something]",
    134: "Taking [something] from [somewhere]",
    135: "Taking [something] out of [something]",
    136: "Tearing [something] into two pieces",
    137: "Tearing [something] just a little bit",
    138: "Throwing [something]",
    139: "Throwing [something] against [something]",
    140: "Throwing [something] in the air and catching it",
    141: "Throwing [something] in the air and letting it fall",
    142: "Throwing [something] onto a surface",
    143: "Tilting [something] with [something] on it slightly so it doesn't fall down",
    144: "Tilting [something] with [something] on it until it falls off",
    145: "Tipping [something] over",
    146: "Tipping [something] with [something] in it over, so [something] falls out",
    147: "Touching (without moving) part of [something]",
    148: "Trying but failing to attach [something] to [something] because it doesn't stick",
    149: "Trying to bend [something unbendable] so nothing happens",
    150: "Trying to pour [something] into [something], but missing so it spills next to it",
    151: "Turning [something] upside down",
    152: "Turning the camera downwards while filming [something]",
    153: "Turning the camera left while filming [something]",
    154: "Turning the camera right while filming [something]",
    155: "Turning the camera upwards while filming [something]",
    156: "Twisting (wringing) [something] wet until water comes out",
    157: "Twisting [something]",
    158: "Uncovering [something]",
    159: "Unfolding [something]",
    160: "Wiping [something] off of [something]",
    161: "Wiping [something] with [something]",
    162: "Holding [something] over [something]",
    163: "Pouring [something] into [something]",
    164: "Pouring [something] out of [something]",
    165: "Pretending to put [something] onto [something]",
    166: "Putting [something] into [something]",
    167: "Putting [something] onto [something]",
    168: "Showing [something] to the camera",
    169: "Tilting [something] with [something] on it until it falls off",
    170: "Approaching [something] with your camera",
    171: "Moving [something] closer to [something]",
    172: "Putting [something] onto [something]",
    173: "Showing [something] to the camera"
}

# Now map your predictions to actual labels
top_indices_from_your_video = [16, 6, 54, 38, 20]  # From your output

print("Your video predictions with actual class names:")
for i, idx in enumerate(top_indices_from_your_video, 1):
    print(f"{i}. Class {idx}: {ssv2_labels[idx]}")

Your video predictions with actual class names:
1. Class 16: Holding [something]
2. Class 6: Covering [something] with [something]
3. Class 54: Pouring [something] into [something]
4. Class 38: Moving [something] down
5. Class 20: Holding [something] over [something]
